In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

# Configurar rutas
current_dir = os.getcwd()
total_data_path = os.path.join(current_dir, "../data/raw/total_data.csv")
output_path = os.path.join(current_dir, "../data/csv_model/validation_xgb_optimized_monthly.csv")

# Parámetros iniciales
chunk_size = 100_000
min_items_per_store = 20
min_sales_threshold = 0.2
min_median_sales = 100
min_sales_value = 20
max_cv = 1.0

# Cargar y procesar datos
monthly_data = []
for chunk in pd.read_csv(total_data_path, chunksize=chunk_size):
    chunk['date'] = pd.to_datetime(chunk['date'])
    chunk['month'] = chunk['date'].dt.to_period('M')
    chunk_monthly = chunk.groupby(['item_id', 'store_id', 'month']).agg({
        'sales': 'sum',
        'event_name_1': 'first',
        'snap_CA': 'first',
        'snap_TX': 'first',
        'snap_WI': 'first',
        'sell_price': 'mean'
    }).reset_index()
    monthly_data.append(chunk_monthly)

df_monthly = pd.concat(monthly_data, ignore_index=True)
df_monthly = df_monthly.groupby(['item_id', 'store_id', 'month']).agg({
    'sales': 'sum',
    'event_name_1': 'first',
    'snap_CA': 'first',
    'snap_TX': 'first',
    'snap_WI': 'first',
    'sell_price': 'mean'
}).reset_index()
df_monthly['month'] = df_monthly['month'].dt.to_timestamp()

# Filtrar series predecibles (mismo proceso que antes)
predictability_metrics = []
for (item_id, store_id), group in df_monthly.groupby(['item_id', 'store_id']):
    sales = group['sales']
    if len(sales) >= 24:
        non_zero_months = (sales > 0).mean()
        mean_sales = sales.mean()
        median_sales = sales.median()
        min_sales = sales.min()
        std_sales = sales.std()
        cv_sales = std_sales / mean_sales if mean_sales > 0 else float('inf')
        predictability_metrics.append({
            'item_id': item_id,
            'store_id': store_id,
            'non_zero_months': non_zero_months,
            'median_sales': median_sales,
            'min_sales': min_sales,
            'cv_sales': cv_sales
        })

metrics_df = pd.DataFrame(predictability_metrics)
filtered_metrics = metrics_df[
    (metrics_df['non_zero_months'] >= min_sales_threshold) &
    (metrics_df['median_sales'] >= min_median_sales) &
    (metrics_df['min_sales'] >= min_sales_value) &
    (metrics_df['cv_sales'] <= max_cv)
]
selected_items = filtered_metrics.groupby('store_id').apply(lambda x: x.nlargest(min_items_per_store, 'median_sales')).reset_index(drop=True)

df_selected = df_monthly[
    df_monthly.set_index(['item_id', 'store_id']).index.isin(
        selected_items.set_index(['item_id', 'store_id']).index
    )
]

# Agregar características
df_selected['event_name_1'] = df_selected['event_name_1'].notnull().astype(int)
df_selected['snap'] = df_selected.apply(
    lambda x: x['snap_CA'] if 'CA' in x['store_id'] else (x['snap_TX'] if 'TX' in x['store_id'] else x['snap_WI']),
    axis=1
)
df_selected['sell_price'] = df_selected['sell_price'].fillna(df_selected['sell_price'].mean())
for lag in [1, 2, 3, 6, 12]:
    df_selected[f'lag_{lag}'] = df_selected.groupby(['item_id', 'store_id'])['sales'].shift(lag)
df_selected['rolling_mean_3'] = df_selected.groupby(['item_id', 'store_id'])['sales'].shift(1).rolling(window=3).mean()
df_selected['month_num'] = df_selected['month'].dt.month
df_selected['year'] = df_selected['month'].dt.year
df_selected = pd.get_dummies(df_selected, columns=['month_num'], prefix='month')
df_selected = df_selected.dropna()

# Preparar predicciones
predictions = []
real_values = []
item_store_combinations = []
months = []

for (item_id, store_id), group in df_selected.groupby(['item_id', 'store_id']):
    group['sales_log'] = np.log1p(group['sales'])
    features = ['event_name_1', 'snap', 'sell_price', 'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12', 'rolling_mean_3', 'year'] + [col for col in group.columns if col.startswith('month_')]
    X = group[features]
    y = group['sales_log']
    X_train, y_train = X.iloc[:-1], y.iloc[:-1]
    X_test, y_test = X.iloc[-1:], y.iloc[-1]
    real = np.expm1(y_test)

    # Optimizar hiperparámetros con Grid Search
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [5, 10],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
    model = XGBRegressor(random_state=42, n_jobs=-1)
    grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    pred_log = best_model.predict(X_test)[0]
    pred = np.expm1(pred_log)

    predictions.append(pred)
    real_values.append(real)
    item_store_combinations.append(f"{item_id}_{store_id}")
    months.append(group['month'].iloc[-1])

# Guardar resultados
comparison_df = pd.DataFrame({
    'item_store': item_store_combinations,
    'month': months,
    'real_sales': real_values,
    'predicted_sales': predictions
})
comparison_df.to_csv(output_path, index=False)

# Calcular métricas
mse = mean_squared_error(real_values, predictions)
mae = mean_absolute_error(real_values, predictions)
r2 = r2_score(real_values, predictions)

print("\n📊 Evaluación del modelo (XGBoost Optimizado, mensual):")
print(f"➡️  MSE:  {mse:.2f}")
print(f"➡️  MAE:  {mae:.2f}")
print(f"➡️  R²:   {r2:.4f}")
print("\nPrimeras 5 filas de los resultados:")
print(comparison_df.head())

/var/folders/js/4g9f6f7d2vj886wxt_jkqkyc0000gn/T/ipykernel_11168/2083847098.py:23: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(total_data_path, chunksize=chunk_size):
/var/folders/js/4g9f6f7d2vj886wxt_jkqkyc0000gn/T/ipykernel_11168/2083847098.py:23: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(total_data_path, chunksize=chunk_size):
/var/folders/js/4g9f6f7d2vj886wxt_jkqkyc0000gn/T/ipykernel_11168/2083847098.py:23: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(total_data_path, chunksize=chunk_size):
/var/folders/js/4g9f6f7d2vj886wxt_jkqkyc0000gn/T/ipykernel_11168/2083847098.py:23: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(total_data_path, chunksi


📊 Evaluación del modelo (XGBoost Optimizado, mensual):
➡️  MSE:  32032.87
➡️  MAE:  125.73
➡️  R²:   0.6690

Primeras 5 filas de los resultados:
         item_store      month  real_sales  predicted_sales
0  FOODS_1_018_CA_1 2016-05-01       206.0       239.411560
1  FOODS_1_018_WI_1 2016-05-01       252.0       220.643784
2  FOODS_1_046_TX_2 2016-05-01       135.0       188.398010
3  FOODS_1_046_WI_1 2016-05-01       160.0       231.337311
4  FOODS_1_085_CA_1 2016-05-01       118.0       288.231323
